# DEA Coastlines generation using command line tools <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">

This notebook demonstrates how to run a DEA Coastlines analysis using command line tools.

### Setup

Update required packages:

In [ ]:
# pip install -r ../requirements.txt

Set working directory to top level of repo to ensure links work correctly:

In [1]:
cd ..

/home/jovyan/Robbi/dea-coastlines


Set analysis parameters:

In [2]:
config_path = 'configs/deafrica_coastlines_config.yaml'
study_area = 87
raster_version = 'testing'
vector_version = 'testing'
continental_version = 'testing'

### Run DEA Coastlines analysis
#### Run tidally-constrained raster generation

In [ ]:
!python -m dea_coastlines.raster --help

Example analysis:

In [ ]:
!python -m dea_coastlines.raster --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1998 --end_year 2021

#### Run vector annual shoreline and rates of change statistics generation

In [ ]:
!python -m dea_coastlines.vector --help

Example analysis:

In [ ]:
!python -m dea_coastlines.vector --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --vector_version {vector_version} --baseline_year 2020

#### Run continental-scale layer generation

In [ ]:
!python -m dea_coastlines.continental --help

Example analysis:

In [ ]:
!python -m dea_coastlines.continental --vector_version {vector_version} --continental_version {continental_version} --shorelines True --ratesofchange True --hotspots True

## Example analysis

In [ ]:
for study_area in [39, 45, 46, 59, 60, 61, 83, 84, 85, 86, 87, 88, 113, 114, 
                   115, 116, 117, 118, 119, 120, 135, 136, 137, 138, 139]:
    !python -m dea_coastlines.raster --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1998 --end_year 2021
    !python -m dea_coastlines.vector --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --vector_version {vector_version} --baseline_year 2020

<Client: 'tcp://127.0.0.1:40277' processes=1 threads=62, memory=512.40 GB>
Parallelising 63 processes
Processing 19984:00                                                             
Processing 1999
Processing 2000
Processing 2001
Processing 2002
Processing 2003
Processing 2004
Processing 2005
Processing 2006
Processing 2007
Processing 2008
Processing 2009
Processing 2010
Processing 2011
Processing 2012
Processing 2013
Processing 2014
Processing 2015
Processing 2016
Processing 2017
Processing 2018
Processing 2019
Processing 2020
Processing 2021
Comparing annual movements with time
Comparing annual movements with soi
<Client: 'tcp://127.0.0.1:42513' processes=1 threads=62, memory=512.40 GB>
Parallelising 63 processes
Processing 19984:00                                                             
CPLReleaseMutex: Error = 1 (Operation not permitted)
Processing 1999
Processing 2000
Processing 2001
Processing 2002
Processing 2003
Processing 2004
Processing 2005
Processing 2006
Processing 2

### Clean up and remove files

In [3]:
rm -rf data/interim/raster/{raster_version}

In [4]:
rm -rf data/interim/vector/{vector_version}

In [ ]:
rm -rf data/processed/{continental_version}